In [1]:
import random
import time
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver import DesiredCapabilities
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException

In [9]:
chrome_options = ChromeOptions()
chrome_options.headless = True
# chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--headless")  # for Jenkins
chrome_options.add_argument("--disable-dev-shm-usage")  # Jenkins
chrome_options.add_argument("--disable-setuid-sandbox")
chrome_options.add_argument("--allow-insecure-localhost")
chrome_options.add_argument("--no-cache")
# chrome_options.add_argument("--user-data-dir=/tmp/user-data")
# chrome_options.add_argument("--hide-scrollbars")
chrome_options.add_argument("--enable-logging")
# chrome_options.add_argument("--log-level=0")
chrome_options.add_argument("--single-process")
# chrome_options.add_argument("--data-path=/tmp/data-path")
chrome_options.add_argument("--ignore-certificate-errors")
# chrome_options.add_argument("--homedir=/tmp")
# chrome_options.add_argument("--disk-cache-dir=/tmp/cache-dir")
# chrome_options.add_argument("user-agent={}".format(CHROME_USER_AGENT))
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--window-size=1900,1000")
chrome_options.add_argument("--disable-software-rasterizer")
chrome_options.add_argument("--ignore-certificate-errors-spki-list")
chrome_options.add_argument("--ignore-ssl-errors")
# chrome_options.add_argument("--crash-dumps-dir=/tmp")
chrome_options.add_argument("--profile-directory=Default")
# chrome_options.add_argument("--disable-infobars")
capabilities = DesiredCapabilities.CHROME.copy()
capabilities["pageLoadStrategy"] = "normal"
chrome_options._caps = capabilities

browser = webdriver.Chrome(
    options=chrome_options, desired_capabilities=capabilities
)

# https://scrapfly.io/blog/how-to-avoid-web-scraping-blocking-javascript/
script = "Object.defineProperty(navigator, 'webdriver', {get: () => false})"
browser.execute_cdp_cmd(
    "Page.addScriptToEvaluateOnNewDocument", {"source": script}
)
dummy_user_agent = {
    # usual user agent string
    "userAgent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "platform": "Linux x86_64",
    "language": "en-US",
    "acceptLanguage": "en-US, en",
    "userAgentMetadata": {
        # ensure the order of this array matches real browser!
        "brands": [
            # at the time of writing this is always == 99
            {"brand": " Not/A)Brand", "version": "99"},
            # ensure that the versions here match ones from User-Agent string
            {"brand": "Google Chrome", "version": "115"},
            {"brand": "Chromium", "version": "115"},
        ],
        "fullVersion": "115.0.0.0",
        "platform": "Linux",
        "platformVersion": "",
        "architecture": "x86_64",
        "model": "",
        "mobile": False,
        "vendor": "Google Inc.",
        "VendorSub": "",
    },
    "product": "Gecko",
    "productSub": "20030107",
}
browser.execute_cdp_cmd("Network.setUserAgentOverride", dummy_user_agent)

In [3]:
def one_letter_at_a_time(delay: float):
    # version 1: send leter by letter
    words = browser.find_element(By.ID, "words").find_elements(By.CLASS_NAME, "word")
    try:
        while len(words) != 0:
            active_index = [i for i in range(len(words)) if 'active' in words[i].get_attribute('class')][0]
            words = words[active_index:]
            all_letters = ""
            for word in words:
                letters = word.find_elements(By.TAG_NAME, "letter")
                for letter in letters:
                    all_letters += letter.text
                all_letters += " "
            for letter in all_letters:
                ActionChains(browser).send_keys(letter).perform()
                time.sleep(delay)
            words = browser.find_element(By.ID, "words").find_elements(By.CLASS_NAME, "word")
    except Exception as e:
        print(e)
    print("Game Over")

In [4]:
def all_letters_at_a_time(delay: float):
    # version 2: send all letters at once
    words = browser.find_element(By.ID, "words").find_elements(By.CLASS_NAME, "word")
    try:
        while len(words) != 0:
            active_index = [i for i in range(len(words)) if 'active' in words[i].get_attribute('class')][0]
            words = words[active_index:]
            all_letters = ""
            for word in words:
                letters = word.find_elements(By.TAG_NAME, "letter")
                for letter in letters:
                    all_letters += letter.text
                all_letters += " "
            ActionChains(browser).send_keys(all_letters).perform()
            time.sleep(delay)
            words = browser.find_element(By.ID, "words").find_elements(By.CLASS_NAME, "word")
    except Exception as e:
        print(e)
    print("Game Over")

In [5]:
def one_word_at_a_time(delay: float):
    # version 3: detect one active word at a time and input (fastest way)
    try:
        while len(browser.find_elements(By.CLASS_NAME, "word")) != 0:
            ActionChains(browser).send_keys([letter.text for letter in browser.find_element(By.CSS_SELECTOR, ".word.active").find_elements(By.TAG_NAME, "letter")] + [' ']).perform()
            time.sleep(delay)
    except Exception as e:
        print(e)
    print("Game Over")

In [6]:
def one_word_letter_at_a_time(delay: float):
    # version 4: detect one active word at a time and input its letter one by one to mimic the most natural way of typing
    try:
        while len(browser.find_elements(By.CLASS_NAME, "word")) != 0:
            active_word = browser.find_element(By.CSS_SELECTOR, ".word.active")
            letters = [letter.text for letter in active_word.find_elements(By.TAG_NAME, "letter")] + [' ']
            for letter in letters:
                ActionChains(browser).send_keys(letter).perform()
                time.sleep(delay)
    except Exception as e:
        print(e)
    print("Game Over")

In [ ]:
def one_word_letter_at_a_time_random(delay: float):
    # version 5: detect one active word at a time and input its letter one by one + randomize delay to mimic the most natural way of typing
    try:
        while len(browser.find_elements(By.CLASS_NAME, "word")) != 0:
            active_word = browser.find_element(By.CSS_SELECTOR, ".word.active")
            letters = [letter.text for letter in active_word.find_elements(By.TAG_NAME, "letter")] + [' ']
            for letter in letters:
                ActionChains(browser).send_keys(letter).perform()
                random_delay = delay * random.random() + 0.005
                time.sleep(random_delay)
    except Exception as e:
        print(e)
    print("Game Over")

In [10]:
url = 'https://monkeytype.com/'
browser.get(url)

# manually click in browser to get ready (accept cookies, choose 15/30/60 seconds, etc.)

In [12]:
delay = 0.01

# uncomment one of the following lines to test different methods
# one_letter_at_a_time(delay)
# all_letters_at_a_time(delay)
# one_word_at_a_time(delay)
# one_word_letter_at_a_time(delay)
one_word_letter_at_a_time_random(delay)

Message: stale element reference: element is not attached to the page document
  (Session info: chrome=104.0.5112.79)
Stacktrace:
0   chromedriver                        0x0000000100929d14 chromedriver + 3792148
1   chromedriver                        0x00000001008c0828 chromedriver + 3360808
2   chromedriver                        0x00000001005c10b8 chromedriver + 217272
3   chromedriver                        0x00000001005c3ae8 chromedriver + 228072
4   chromedriver                        0x00000001005c3950 chromedriver + 227664
5   chromedriver                        0x00000001005c3b84 chromedriver + 228228
6   chromedriver                        0x00000001005ec0f0 chromedriver + 393456
7   chromedriver                        0x00000001005e7614 chromedriver + 374292
8   chromedriver                        0x0000000100619a60 chromedriver + 580192
9   chromedriver                        0x00000001005e6318 chromedriver + 369432
10  chromedriver                        0x00000001008ff1e8

In [14]:
wpm = browser.find_element(By.CSS_SELECTOR, ".group.wpm").find_element(By.CLASS_NAME, "bottom").text
acc = browser.find_element(By.CSS_SELECTOR, ".group.acc").find_element(By.CLASS_NAME, "bottom").text
consistency = browser.find_element(By.CSS_SELECTOR, ".group.flat.consistency").find_element(By.CLASS_NAME, "bottom").text
print("wpm: " + wpm)
print("accuracy: " + acc)
print("consistency: " + consistency)

wpm: 409
accuracy: 100%
consistency: 94%


In [ ]:
browser.close()